Farneback Algoritması (Dense Optical Flow)

In [30]:
import cv2 as cv
import numpy as np

In [31]:
# Göreceli yol ile video dosyasına erişim
video_path = "../04_VIDEO_ANALYSIS_BACKGROUND_SUBTRACTION_AND_ROI_EXTRACTION_OF_THE_FOREGROUND/pedestrian.mp4"
# VideoCapture nesnesi ile video dosyasını aç
cap = cv.VideoCapture(video_path)

In [32]:
ret, frame1 = cap.read() # İlk kareyi oku

prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY) # İlk kareyi gri tonlamaya çevir

In [33]:
# HSV formatında boş bir görüntü matrisi oluştur ve 
# doygunluk kanalını (S) 255'e ayarla
hsv = np.zeros_like(frame1)
hsv[..., 1] = 255

In [34]:
def dense_opt_flow(hsv, prvs):
    while 1: # Sonsuz döngü
        ret, frame2 = cap.read() # Bir sonraki kareyi oku

        if not ret: # Eğer kare alınamadıysa döngüden çık
            print("Video sonuna ulaşıldı veya dosya okunamıyor.")
            break
                
        nextt = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY) # Yeni kareyi gri tonlamaya çevir
        
        # Farneback algoritması ile optik akışı hesapla
        flow = cv.calcOpticalFlowFarneback(
            prvs,       # 'prvs': Önceki kare (gri tonlamalı).
            nextt,      # 'nextt': Mevcut kare (gri tonlamalı).
            None,       # None: Optik akış hesaplamaları için boş matris (ilk değer).
            0.5,        # 'pyr_scale': Görüntü piramidi skalası (küçültme oranı), 0.5 = her seviye yarı boyutta.
            3,          # 'levels': Piramitteki seviye sayısı (katman sayısı).
            15,         # 'winsize': Optik akışın hesaplanacağı pencere boyutu (büyüklüğü pikseller cinsinden).
            3,          # 'iterations': Her piramit seviyesi için algoritmanın kaç iterasyon yapacağı.
            5,          # 'poly_n': Optik akışın polinomsal genişletme derecesi (genişleme çevresindeki alan).
            1.2,        # 'poly_sigma': Gaussian kerneli için sigma değeri (filtreleme pürüzsüzleştirme).
            0)           # 'flags': Ekstra işleme bayrakları, burada sıfır (standart işlem).        
        
        # Akış vektörlerini polar koordinatlara çevir (büyüklük ve açı)
        mag, ang = cv.cartToPolar(flow[..., 0], 
                                  flow[..., 1])
        # Açıyı HSV formatının Hue (renk tonu) kanalına aktar
        hsv[..., 0] = ang*180/np.pi/2
        # Büyüklüğü normalleştir ve HSV'nin parlaklık (V) kanalına aktar
        hsv[..., 2] = cv.normalize(mag,        # 'mag': Normalleştirilecek büyüklük verisi (optik akışın büyüklüğü).
                                   None,       # None: Çıkış matrisi, burada None kullanılıyor (yeni bir matris oluşturulacak).
                                   0,          # 'alpha': Normalleştirilen değerlerin minimum sınırı.
                                   255,        # 'beta': Normalleştirilen değerlerin maksimum sınırı.
                                   cv.NORM_MINMAX) # 'norm_type': Normalizasyon tipi, burada minimum ve maksimuma göre normalize edilir.
            
        # HSV'yi geri BGR formatına çevir
        bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
        
        # Sonraki kareyi ve optik akış sonuçlarını görüntüle
        cv.imshow("frame2", bgr)
        cv.imshow("frame1", frame2)
        
        # 'Esc' tuşuna basarak döngüyü sonlandır
        k = cv.waitKey(30) & 0xff
        if k == 27:
            break

        prvs = nextt # Sonraki kareyi bir önceki kare olarak ayarla

In [35]:
dense_opt_flow(hsv, prvs) # Fonksiyonu çağırarak optik akış işlemini başlat

Video sonuna ulaşıldı veya dosya okunamıyor.


In [36]:
cv.destroyAllWindows() # Tüm pencereleri kapat

![](ss_of_videos_3.png)